In [3]:
import hid
import struct
import time

    # BUTTON_MAP = {
    #     0:  "DPAD_UP",
    #     1:  "DPAD_DOWN",
    #     2:  "DPAD_LEFT",
    #     3:  "DPAD_RIGHT",
    #     4:  "MENU",      # "Start" button
    #     5:  "VIEW",      # "Back" button
    #     6:  "LS",        # Left Stick Click
    #     7:  "RS",        # Right Stick Click
    #     8:  "LB",        # Left Bumper
    #     9:  "RB",        # Right Bumper
    #     10: "GUIDE",     # Xbox "Guide" button
    #     11: "SHARE",     # Share/Capture button
    #     12: "A",
    #     13: "B",
    #     14: "X",
    #     15: "Y",
    # }

class XboxController:
    # --- 正确的按钮位映射 for Xbox Series X/S Controller (PID 0x0B12) ---
    # 这个映射基于常见的 HID 报告格式
    BUTTON_MAP = {
        0:  "DPAD_UP",
        1:  "DPAD_DOWN",
        2:  "DPAD_LEFT",
        3:  "DPAD_RIGHT",
        4:  "A",      # "Start" button
        5:  "B",      # "Back" button
        6:  "X",        # Left Stick Click
        7:  "Y",        # Right Stick Click
        8:  "1",
        9:  "2",
        10: "GUIDE",     # Xbox "Guide" button
        11: "SHARE",     # Share/Capture button
        12: "LB",
        13: "RB",
        14: "3",
        15: "4",
    }



    def __init__(self, vendor_id=0x045E, product_id=0x0B12,
                 use_unsigned_axes=False, report_length=19, nonblocking=True):
        """
        :param use_unsigned_axes: 如果设备把轴数据以 uint16 (0..65535, center ~=32768) 发送，
                                 设置为 True；否则使用 int16（默认 False）。
        """
        self.vendor_id = vendor_id
        self.product_id = product_id
        self.use_unsigned_axes = use_unsigned_axes
        self.report_length = report_length # 报告长度至少需要 18 for sticks

        self.device = hid.device()
        self.device.open(vendor_id, product_id)
        if nonblocking:
            self.device.set_nonblocking(True)

        print("Connected:", self.device.get_manufacturer_string(),
              self.device.get_product_string())

    def close(self):
        try:
            self.device.close()
        except Exception:
            pass

    def read(self):
        """
        读取并解析一次报告，返回 dict 或 None（无数据）。
        """
        data = self.device.read(64)
        if not data:
            return None
        # 报告长度需要至少 18 才能包含所有摇杆数据
        if len(data) < 18:
            return None

        raw = bytes(data)

        # ----- 修正后的解析逻辑 -----
        report_id = raw[0]
        # 跳过 1-3 字节 (序列号等)
        
        # 按钮在字节 4 和 5
        buttons_raw = raw[4] + (raw[5] << 8)
        
        # 扳机键在字节 6-9，是 10 位无符号数 (0-1023)
        # struct.unpack_from 返回一个元组，即使只有一个元素
        lt, rt = struct.unpack_from("<HH", raw, 6)

        # 摇杆在字节 10-17，这部分原始代码是正确的
        if self.use_unsigned_axes:
            ax0, ax1, ax2, ax3 = struct.unpack_from("<HHHH", raw, 10)
            lx = self._u16_to_signed(ax0)
            ly = self._u16_to_signed(ax1) # Y 轴通常是反的
            rx = self._u16_to_signed(ax2)
            ry = self._u16_to_signed(ax3) # Y 轴通常是反的
        else:
            lx, ly, rx, ry = struct.unpack_from("<hhhh", raw, 10)
        
        # 通常 HID 报告中 Y 轴向上为负，这里进行反转以符合常见游戏习惯（上为正）
        ly, ry = -ly, -ry

        return {
            "raw_bytes": list(raw[:self.report_length]),
            "report_id": report_id,
            "buttons_raw": buttons_raw,
            "buttons": self._decode_buttons(buttons_raw),
            "lt": lt,
            "rt": rt,
            "lx": lx,
            "ly": ly,
            "rx": rx,
            "ry": ry,
            "lt_norm": self._normalize_trigger(lt),
            "rt_norm": self._normalize_trigger(rt),
            "lx_norm": self._normalize_axis(lx),
            "ly_norm": self._normalize_axis(ly),
            "rx_norm": self._normalize_axis(rx),
            "ry_norm": self._normalize_axis(ry),
        }

    def _decode_buttons(self, bitmask):
        """使用正确的按钮位映射解码"""
        return {name: bool(bitmask & (1 << bit)) for bit, name in self.BUTTON_MAP.items()}

    def _u16_to_signed(self, v):
        """把 uint16 (0..65535, 中心约 32768) 转成 signed short 范围（-32768..32767）"""
        # 这个函数逻辑是正确的
        s = int(v) - 32768
        if s > 32767:
            s -= 65536
        return s

    def _normalize_axis(self, v):
        """把 signed short (-32768..32767) 归一化到 -1..1"""
        # 这个函数逻辑是正确的
        v = int(v)
        if v < 0:
            return max(-1.0, v / 32768.0)
        else:
            return min(1.0, v / 32767.0)
            
    def _normalize_trigger(self, v):
        """把 10-bit trigger (0..1023) 归一化到 0..1"""
        return v / 1023.0


if __name__ == "__main__":
    try:
        # 现代 Xbox 控制器通常使用 signed axes
        xbox = XboxController(use_unsigned_axes=False) 
        
        last_print_time = time.time()
        
        while True:
            state = xbox.read()
            if state:
                # 为了避免刷屏，每 100ms 打印一次状态
                current_time = time.time()
                if current_time - last_print_time >= 0.1:
                    # 格式化输出，更易读
                    buttons_pressed = [name for name, pressed in state["buttons"].items() if pressed]
                    print(
                        f"LX: {state['lx_norm']:.2f}, LY: {state['ly_norm']:.2f} | "
                        f"RX: {state['rx_norm']:.2f}, RY: {state['ry_norm']:.2f} | "
                        f"LT: {state['lt_norm']:.2f}, RT: {state['rt_norm']:.2f} | "
                        f"Buttons: {buttons_pressed}"
                    )
                    last_print_time = current_time

            time.sleep(0.005) # 保持一个较小的休眠以降低 CPU 占用

    except OSError as e:
        print(f"无法打开设备，请检查设备是否连接或权限是否正确: {e}")
    except KeyboardInterrupt:
        print("\n退出")
    finally:
        try:
            xbox.close()
        except NameError: # 如果 xbox 对象未成功创建
            pass

Connected: Microsoft Controller
LX: 0.03, LY: 0.00 | RX: -0.02, RY: 0.00 | LT: 0.00, RT: 0.00 | Buttons: []
LX: 0.05, LY: 0.01 | RX: -0.02, RY: 0.00 | LT: 0.00, RT: 0.00 | Buttons: ['LB']
LX: 0.05, LY: 0.01 | RX: -0.02, RY: 0.00 | LT: 0.00, RT: 0.00 | Buttons: []
LX: 0.05, LY: 0.01 | RX: -0.02, RY: 0.00 | LT: 0.00, RT: 0.00 | Buttons: ['RB']
LX: 0.05, LY: 0.01 | RX: -0.02, RY: 0.00 | LT: 0.00, RT: 0.00 | Buttons: []
LX: 0.05, LY: 0.01 | RX: -0.02, RY: 0.00 | LT: 0.00, RT: 0.00 | Buttons: ['LB']
LX: 0.05, LY: 0.01 | RX: -0.02, RY: 0.00 | LT: 0.00, RT: 0.00 | Buttons: []
LX: 0.05, LY: 0.01 | RX: -0.02, RY: 0.00 | LT: 0.00, RT: 0.00 | Buttons: ['RB']
LX: 0.05, LY: -0.00 | RX: -0.02, RY: 0.00 | LT: 0.00, RT: 0.00 | Buttons: ['RB']
LX: 0.05, LY: -0.00 | RX: -0.02, RY: 0.00 | LT: 0.00, RT: 0.00 | Buttons: []
LX: 0.05, LY: -0.00 | RX: -0.02, RY: 0.03 | LT: 0.00, RT: 0.00 | Buttons: []
LX: 0.05, LY: -0.00 | RX: 0.02, RY: 0.01 | LT: 0.00, RT: 0.00 | Buttons: []
LX: 0.05, LY: -0.00 | RX: 0.02, R